<h1 align='center'  style = 'margin-bottom: 0px'> The Pencil _Shaft_ Factory </h1>
<h2 align='center'  style = 'margin-top: 0px'> The Graphical Method for <br/> Linear Programs </h2>
<p>&nbsp;</p>

__Note__: Although the problem is "realistic," the numbers are contrived so as to make the
example accessible and useful for a large number of concepts in the course.  

We begin by __import__ing some Python modules we will use below:  Place your cursor in the cell below (i.e., select the cell), and then execute __[ctrl]-[enter]__.

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import numpy as np 

from ipywidgets import interact

## BE SURE THAT MainExampleTools.py is in same folder as this notebook!
%run -i 1-MainExampleTools.py 

<p>&nbsp;</p>

## The Linear Program
Solving our main example means finding the solution(s) to the following linear program:  
$$ maximize \; z = 2x+3y \quad subject \; to \; (x,y) \in \Omega $$ 

where the feasible region $\Omega$ is given by 
$$ \Omega = \left\lbrace (x,y)\in\mathbb{R}^2 \; \; \middle\vert
      \begin{array}{c} 
        x + \; y  \le  20 \\ 
        x + 2y  \le 30 \\
        x  \le 15 \\ 
        y  \le 2x \\ 
        x \ge 0, \quad y \ge 0
      \end{array}
    \right\rbrace 
$$
Similar to the way in which we stated the problem, there are 2 tasks we must address

1. Determining the structure of the Feasible Region. 

2. Finding where the largest value of $z$ occurs over the Feasible Region. 

Let's now consider the role of the objective function (issue #2). 

In some sense, if we know all the vertices of the Feasible region, then we need only substitute them respectively into the objective and determine which produces the largest value of $z$.    This works great for small examples, but remember that real world applications involves tens, hundreds, thousands, and even millions of variables, and in such contexts, a feasible region can have arbitrarily huge numbers of vertices. 

<div align = "center" style = "font-size:larger; color:blue; margin-top:20px; margin-bottom:20px"> 
Our goal is a method that computes vertices only as long as the objective is increasing. </div>

This method will be called the _Simplex Method_, and its central feature is that by tracking the objective function we can arrive at a solution to a linear program via calculation of only a relatively few of the vertices of the feasible region. 

<p>&nbsp;</p>

## Optimizating an Objective over a Feasible Region 
First off, if $\Omega \subset \mathbb{R}^n$, then the objective as an equation introduces an additional variable $z$.  Moreover, since the objective is a linear function, the linear program is actually asking for a vertex with largest possible $z$ on a polytope in $n+1$ dimensional space.   

For example, when we write the objective function in equation form, such as in  
$$ maximize \; z = 2x+3y \quad instead\;of\quad maximize 2x+3y,$$
we are actually adding a new constraint.  Indeed, in our main example, the linear program 
\begin{eqnarray*} 
maximize   & z = 2x+3y \\
subject \; to & x + \; y  \le  20 \\ 
              & x + 2y  \le 30 \\
              & x  \le 15 \\ 
              & y  \le 2x \\ 
              & x\ge 0, \quad y \ge 0
\end{eqnarray*}
should -- and does -- have exactly the same solution as the linear program 
\begin{eqnarray*} 
maximize   & z  \\
subject \; to & z = 2x+3y  \\ 
              & x + \; y  \le  20 \\ 
              & x + 2y  \le 30 \\
              & x  \le 15 \\ 
              & y  \le 2x \\ 
              & x\ge 0, \quad y \ge 0
\end{eqnarray*}
However, this last linear program is in _3 dimensions_ (i.e., in the 3 independent variables $x,y,z$).  Indeed, we can write it in the form 
\begin{eqnarray*} 
maximize   & z  \\
subject \; to & 2x+3y    & - z & = 0  \\ 
              & x + \; y &     &  \le  20 \\ 
              & x + 2y   &     & \le 30 \\
              & x        &     & \le 15 \\ 
              & -2x + y  &     &  \le 0 \\ 
              & x\ge 0,  &     & y \ge 0
\end{eqnarray*}
The feasible region for this last program, which we will call $\Omega'$, is still a polygon, but in the plane $z = 2x+3y$ rather than in the xy-plane. The objective is to find the vertex of $\Omega'$ with the largest $z$ coordinate, which in our main example is $(10,10,50)$.   

All this is illustrated below: 

In [ ]:
vertices = [ (0,0), (15,0), (15,5), (10,10), (6,12), (0,0) ]
XY = np.array(vertices).T
X,Y = XY[0], XY[1]
Z = 2*X+3*Y

fig = plt.figure(figsize = (8,8))
ax3d = fig.gca(projection='3d')

ax3d.plot_trisurf(X, Y, Z, linewidth=1, antialiased=True, alpha = 0.5, color='cyan', shade = False)
ax3d.plot_trisurf(X, Y, 0*Z, antialiased = True, color = 'cyan', alpha = 0.5)
ax3d.plot_trisurf([0,17,17,0],[0,0,14,14], [0,34,76,42], antialiased = True, color = 'pink', alpha = 0.25, shade = False)

ax3d.set_xlabel("x"); ax3d.set_ylabel("y"); ax3d.set_zlabel("z")
for i in range(len(X)):
    ax3d.plot([X[i],X[i]],[Y[i],Y[i]],[0,Z[i]], color='darkblue', alpha = 0.5)
ax3d.plot(X,Y,   Z, color = 'blue')#, alpha = 0.5)
ax3d.plot(X,Y, 0*Z, color = 'darkblue', alpha = 0.5)
ax3d.scatter([10],[10],[50], color='red')
ax3d.text(5,5,25, "$\Omega'$", fontsize = 14)
ax3d.text(12,12,60, '$z = 2x+3y$', fontsize = 14, color = 'pink')
ax3d.text(10,10,50,'  $(10,10,50)$')
ax3d.text(10,4,0, '$\Omega$', fontsize = 14)
ax3d.set_xlim(0,16); ax3d.set_ylim(0,14);ax3d.set_zlim(0,55)
ax3d.set_title('$(x*,y*,z*) = (10,10,50)$\n', fontsize = 14)


Indeed, in general, if we write the objective in equation form $z = ...$, then we are actually adding a constraint and creating a space that is one dimension higher than the original.  This brings up a very important concept in operations research in general.  

<div align = "center" style="margin-top:20px;margin-bottom:20px;font-size:larger;color:blue"> Solving linear and nonlinear programs often means creating new, often 'artificial' variables <br/> and working in typically much higher dimensional spaces than the original problem.</div>

However, rather than think of a "new" feasible region, $\Omega'$, we instead simply think of the original feasible region $\Omega$ subjected to the objective function $2x+3y$, which maps $\Omega$ into the plane with equation $z = 2x+3y$. 

In [ ]:
vertices = [ (0,0), (15,0), (15,5), (10,10), (6,12), (0,0) ]
XY = np.array(vertices).T
X,Y = XY[0], XY[1]
Z = 2*X+3*Y

fig = plt.figure(figsize = (8,8))
ax3d = fig.gca(projection='3d')

def LPpolyhedron(ax):
    ax.plot_trisurf(X, Y, Z, linewidth=1, antialiased=True, alpha = 0.5, color='cyan', shade = False)
    ax.plot_trisurf(X, Y, 0*Z, antialiased = True, color = 'cyan', alpha = 0.5)
    ax.set_xlabel("x"); ax.set_ylabel("y"); ax.set_zlabel("z")
    for i in range(len(X)):
        ax.plot([X[i],X[i]],[Y[i],Y[i]],[0,Z[i]], color='darkblue', alpha = 0.5)
    ax.plot(X,Y,   Z, color = 'blue')#, alpha = 0.5)
    ax.plot(X,Y, 0*Z, color = 'darkblue', alpha = 0.5)
    ax.scatter([10],[10],[50], color='red')
    ax.text(5,5,40, "$z = 2x+3y$", fontsize = 14)
    ax.text(10,10,50,'  $(10,10,50)$')
    ax.text(12,6,0, '$\Omega$', fontsize = 14)
    ax.set_xlim(0,16); ax.set_ylim(0,14);ax.set_zlim(0,55)
    ax.set_title('$(x*,y*,z*) = (10,10,50)$\n', fontsize = 14)
LPpolyhedron(ax3d)

The advantage of this latter conceptualization of a linear program is that it allows us to work directly with $\Omega$ in the xy-plane, and to think of the added dimensionality in terms of _level curves_ or _isolines_ of the objective rather than as a new object in a higher dimensional context.  

That is, it is better to think of the objective function as a _ __ family of level curves__ _.  For example, the objective 
$ z = 2x+3y$ can be considered to be the set of all lines of the form $$ 2x+3y= k \qquad and \; z = k.$$

_Notice that as z increases, the level curves move from vertex to vertex until a maximum is achieved._

In [ ]:
fig = plt.figure(figsize = (8,8))
ax3d = fig.gca(projection='3d')

LPpolyhedron(ax3d)

inPlane = None
onSurf = None
lbl1, lbl2 = None, None
def LevelCurve( k = 0 ):
    global inPlane, onSurf, lbl1, lbl2
    try:
        ax3d.lines.remove(inPlane[0])
        ax3d.lines.remove(onSurf[0])
        ax3d.texts.remove(lbl1)
        ax3d.texts.remove(lbl2)
    except:
        pass
    if( k < 30):
        xr, yr = k/2, 0
    else:  #( 30 <= k < 55 ):
        xr, yr = 15, k/3 - 10
        
    inPlane = ax3d.plot([k/8,xr],[k/4,yr], color='magenta')
    onSurf  = ax3d.plot([k/8,k/8,xr,xr],[k/4,k/4,yr,yr],[0,k,k,0], color='magenta', linestyle = 'dashed')
    lbl1    = ax3d.text( k/16 + xr/2, k/8 + yr/2, 0, '$2x+3y=%s$' % k, color='magenta')
    lbl2    = ax3d.text( k/8, k/4, k/2, '$z = %s$' % k, color='magenta')

interact(LevelCurve, k=(0,50,1));

Thus, we can replace working with $z=2x+3y$ in 3 dimensions by the family of level curves $$2x+3y=k, \; k\in\mathbb{R}.$$  This allows us to work with the objective function in terms of the feasible region, and in fact, _it will be our preferred and indeed only approach used for visualizing the objective function._ 

Below we have the same visualization as above but restricted to the xy-plane. 

In [ ]:
fig, axes = plt.subplots()

#plot the Feasible Region
vertices = [ (0,0), (15,0), (15,5), (10,10), (6,12) ]
X,Y = zip(*vertices)
axes.fill(X,Y, facecolor='cyan', edgecolor='black', zorder = 1)

#plot the vertices
axes.scatter(X,Y, color='brown', zorder = 2)
for vert in vertices:
    axes.text( vert[0], vert[1], " ({0},{1})".format(vert[0], vert[1]), fontsize = 14)

# Label the resulting feasible region
axes.text(5,3, '$\Omega$', fontsize = 14 )

# Plot the xy-axes and set upper bound on y
axes.set_xlim(0,30)
axes.set_ylim(0,20)
axes.set_xlabel("x = 1000's of Standard shafts\n")
axes.set_ylabel("y = 1000's of Engineering shafts")

Line, Label = None, None
def LevelCurve(k):
    global Line, Label
    try:
        axes.lines.remove(Line[0])
        axes.texts.remove(Label)
    except:
        pass
    
    Line  = axes.plot( [0,k/2], [k/3,0], color='magenta' )
    Label = axes.text( k/4, k/6, ' $2x+3y=%s$' % k, color='magenta', fontsize = 14)
    axes.set_title('$k=%s$' % k, fontsize = 14)
    axes.grid(True)

interact(LevelCurve, k=(0,80,1));

One immediate consequence is that we can use increasing values of $k$ to avoid computing many of the vertices.  For example, we know that $z=0$ at the origin, and as we increase $x$ from 0 to 15, the value of $k$ increases from 0 to 30 (_Try it out!!_). 

For $x=15$, we then increase $y$ from 0 to $5$, which increases $k$ from 30 to 45.  (_Try it out!_). 

We cannot continue, but must instead turn along the constraint $ x + y = 20$, along which $k$ increases from 45 to 50.  

However, if we then start to move along $x+2y = 30$, then $k$ _ __ decreases__ _ from 50 to 48.  

Conversely, as $k$ increases, the line $2x+3y=k$ leaves the feasible region. Thus, the answer is $z^* = 50$ corresponding to 
$$ argmax(z) = (10,10) $$
which is to say that $(x^*,y^*) = (10,10)$. 

### FINAL NOTES
The _Simplex method_ is the default method for GLPK.  Below you can see that it arrives at the same maximum value for $z$ as we do, namely $$ z = 50 = 5.000000000e+01.$$
Also notice that technically, we do not define $x \le 15$ as a constraint, but rather we _restrict_ the domain of the variable $x$ to the interval [0,15].    

In [ ]:
%%script glpsol -m /dev/stdin

# declare problem variables
var x >= 0, <= 15 ;  # we can define properties for
var y >= 0 ;  # the variables as we define them

# declare the objective 
maximize z: 2*x + 3*y;       ## DON'T FORGET THOSE SEMICOLONS!!!!! 

# subject to the constraints
s.t.    constraint1:    x +   y <=  20;
        constraint2:    x + 2*y <=  30;
        constraint3:          y <= 2*x; 

# solve
solve;

# display results
display x, y, z;

end;

There are better computational methods than the Simplex method, but we still require students to learn the Simplex method because it provides us intuition and insight into the nature of linear programs.  

But first, we need to introduce some linear algebra notation and concepts (there are linear systems to be solved, after all!). 

<p>&nbsp;</p>

## Additional Examples 

Until we have sufficient linear algebra, we'll restrict our attention to 2 dimensional feasible regions and use a "graphical method" in the spirit of the simplex method.  That is, we will 

1. Calculate a starting vertex using a pair of constraints. 
2. Compute an adjacent vertex corresponding to a higher level of the objective. 
3. Continue until the objective cannot be increased any further within the feasible region. 

Often -- but not always -- we can use the origin as the starting vertex.   

<b style = "font-size:larger">Example 2 :</b> Solve the following linear program using our "graphical method": 
\begin{eqnarray*}
maximize \;  & z = 2x + 5y \\
subject \; to \; &      x + y \le 25 \\
       & 2x + y \le 30 \\
       & x + 3y \le 45 \\
       & x \ge 0, y \ge 0
\end{eqnarray*} 
We found the feasible region in the last lecture, including its vertices, which we recall below.  

In [ ]:
fig, axes = plt.subplots()

# The Feasible Region
vertices = np.array( [ (0,0), (0,15), (9,12), (15,0)] ).T
axes.plot([0,25],[25,0], 'k--')
axes.fill(*vertices, facecolor = 'cyan', edgecolor = 'black', zorder = 0 )
axes.scatter(*vertices, color = 'brown', zorder = 1)
for vertex in vertices.T: 
     axes.text( *vertex, " ({0},{1})".format(*vertex),zorder = 1)
axes.text(4,14,'$x+3y = 45$')
axes.text(12,7, '$2x+y = 30$')
axes.text(10,15, ' $x+y = 25$')

# Label the resulting feasible region
axes.text(5,3, '$\Omega$', fontsize = 14,  zorder = 1 )

# restrict to first quadrant 
axes.set_xlim(0,)
axes.set_ylim(0,)

Let's focus on the feasible region and notice that the objective $z = 2x + 5y$ has a level of $k=0$ at the origin.  But if we move to the vertex (15,0), then $k = 30$, and correspondingly, it lies on the level curve $2x+5y = 30$. 

In [ ]:
fig, axes = plt.subplots()

# Plot Level Curves
x = np.linspace(0,15,5)
for k in [10, 20, 30]:
    y = (k-2*x)/5
    axes.plot( x,y,color='magenta'  )
axes.text( x[2],y[2], ' $2x+5y = %s$' % k, color = 'magenta', fontsize = 14)

# The Feasible Region
vertices = np.array( [ (0,0), (0,15), (9,12), (15,0)] ).T
axes.fill(*vertices, facecolor = 'cyan', edgecolor = 'black', zorder = 0 )
axes.scatter(*vertices, color = 'brown', zorder = 1)
for vertex in vertices.T: 
     axes.text( *vertex, " ({0},{1})".format(*vertex),zorder = 1)
axes.text(4,14,'$x+3y = 45$')
axes.text(12,7, '$2x+y = 30$')
axes.text(10,15, ' $x+y = 25$')

# Label the resulting feasible region
axes.text(5,3, '$\Omega$', fontsize = 14,  zorder = 1 )

# restrict to first quadrant 
axes.set_xlim(0,)
axes.set_ylim(0,)

Moving to the vertex (9,12) increases the level to $k = 78$, and correspondingly, it lies on the level curve $2x+5y = 78$. 

In [ ]:
for k in [45, 60, 78]:
    y = (k-2*x)/5
    axes.plot( x,y,color='magenta'  )
axes.text( x[2],y[2], ' $2x+5y = %s$' % k, color = 'magenta', fontsize = 14)
fig

Any higher levels of $k$ yield _level curves_ ( aka, _isolines_ ) that do not intersect the feasible region.  Hence, the maximum value is $z^* = 78$ and occurs at the argmax of $(x^*,y^*) = (9,12)$. 

<p>&nbsp;</p>
<b style = "font-size:larger">Example 3 :</b> Solve the following linear program using our "graphical method": 
\begin{eqnarray*}
maximize \;  & z = 2x + 3y  \\
subject \; to \;   &  x + y \le 20 \\
                   & 2x + y \le 25 \\
                   & x \le 10 \\
                   & x \ge 0, y \ge 0
\end{eqnarray*} 
First, let's graph the region. 

In [ ]:
fig, axes = plt.subplots()
add_method(axes, plotfill)

#plot the feasible region satisfying the constraint
x = np.linspace(0,10,20)
axes.plotfill( x, dict(a=1,b=1,c=20), color = 'blue', label = '$x+y \leq 20$' )
axes.plotfill( x, dict(a=2,b=1,c=25), color = 'green', label = '$2x+y \leq 25$' )

# plot the initial vertex 
axes.scatter([0],[0], color = 'brown', zorder = 10)
axes.set_xlim(0,)
axes.set_ylim(0,)
axes.legend()

The objective $z = 2x+3y$ has a level of $k=0$ at the origin.  But if we move to the vertex (10,0), then $k = 20$, and correspondingly, it lies on the level curve $2x+3y = 20$. 

In [ ]:
for k in [5,10,15,20]:
    y = (k-2*x)/3
    axes.plot( x,y,color='magenta'  )
axes.scatter([10],[0], color = 'brown', zorder = 10)
axes.text( x[2],y[2], ' $2x+3y = %s$' % k, color = 'magenta', fontsize = 14)
fig

The next vertex corresponds to the intersection of $x=10$ and $2x+y = 25$, which implies $y=5$ and the vertex (10,5).  

If we move to the vertex (10,5), then $k = 35$, and correspondingly, it lies on the level curve $2x+3y = 35$. 

In [ ]:
for k in [25,30,35]:
    y = (k-2*x)/3
    axes.plot( x,y,color='magenta'  )
axes.text( x[2],y[2], ' $2x+3y = %s$' % k, color = 'magenta', fontsize = 14)
axes.scatter([10],[5], color = 'brown', zorder = 10)
fig

Clearly, we can keep increasing the levels of the objective (i.e., we can choose higher values of $k$), and the next vertex is at the intersection of $x+y = 20$ and $2x+y = 25$, which is at $(5,15)$.  

If we move to the vertex (5,15), then $k = 55$, and correspondingly, it lies on the level curve $2x+3y = 55$.

In [ ]:
for k in [41,48,55]:
    y = (k-2*x)/3
    axes.plot( x,y,color='magenta'  )
axes.text( x[2],y[2], ' $2x+3y = %s$' % k, color = 'magenta', fontsize = 14)
axes.scatter([5],[15], color = 'brown', zorder = 10)
fig

Again, we can keep increasing the levels of the objective (i.e., we can choose higher values of $k$), and the next vertex is at $(0,20)$, the y-intercept of the line $x+y=20$.  

If we move to the vertex $(0,20)$, then $k = 60$, and correspondingly, it lies on the level curve $2x+3y = 60$.

In [ ]:
for k in [60]:
    y = (k-2*x)/3
    axes.plot( x,y,color='magenta'  )
axes.text( x[2],y[2], ' $2x+3y = %s$' % k, color = 'magenta', fontsize = 14)
axes.scatter([0],[20], color = 'brown', zorder = 10)
fig

Thus, the answer is that a maximum of $z^* = 60$ occurs at the argmax $(x^*,y^*) = (0, 20)$.  

We in fact in this example calculated every vertex, but that was only because we were not clever.  In practice, GLPK solves this in only one step.  

In [ ]:
%%script glpsol -m /dev/stdin

# declare problem variables
var x >= 0, <= 10 ;  # we can define properties for
var y >= 0 ;  # the variables as we define them

# declare the objective 
maximize z: 2*x + 3*y;       ## DON'T FORGET THOSE SEMICOLONS!!!!! 

# subject to the constraints
s.t.    constraint1:    x +  y <= 20;
        constraint2:  2*x +  y <= 25; 

# solve
solve;

# display results
display x, y, z;

end;

In particular, glpsol output should contain the two lines 
        *     0: obj =  -0.000000000e+00 inf =   0.000e+00 (2)
        *     1: obj =   6.000000000e+01 inf =   0.000e+00 (0)
The first line says that initially, glpsol is at the vertex with an objective value of 0, which means it starts at the origin.  
The next line says that the $1^st$ iteration goes to the vertex with an objective value of 

__ obj = 6.000000000e+01__   which is 60

However, our focus will be on learning the simplex method for its pedagogical value, and then we will simply trust that -- as usual -- the actual algorithm that is implemented has been tweaked and tuned for optimal performance.  Correspondingly, we will not try to visit all the ways that the simplex method can be optimized for speed and accuracy.  

<b style = "font-size:larger">Example 3 :</b> Solve the following linear program using our "graphical method": 
\begin{eqnarray*}
minimize \;  & z = 4x+2y  \\
subject \; to \;   & 5x + y  \ge 25 \\
                   & 3x + 2y \ge 30 \\
                   & x -  2y \le 5 \\
                   & x \ge 0, y \ge 0
\end{eqnarray*} 
First, let's graph the region. Notice that if (x,y) = (0,0), then the 3rd constraint, $x-2y\le 5$ is true.  Thus, we fill the side of the line $x - 2y = 5$ that contains the origin, and the origin is __above__ the line.  

In [ ]:
fig, axes = plt.subplots()
add_method(axes, plotfill)

#plot the feasible region satisfying the constraint
x = np.linspace(0,20,20)
axes.plotfill( x, dict(a=5,b= 1,c=25), region = 'above', color = 'blue', hatched = True, label = '$5x+y \geq 25$' )
axes.plotfill( x, dict(a=3,b= 2,c=30), region = 'above', color = 'green', hatched = True, label = '$3x+2y \geq 30$' )
axes.plotfill( x, dict(a=1,b=-2,c=5), region = 'above', color = 'red', hatched = True, label = '$x - 2y \leq 5$')

# plot the initial vertex 
axes.scatter([0],[25], color = 'brown', zorder = 10)
axes.set_xlim(0,)
axes.set_ylim(0,30)
axes.legend()

The problem here is that there is not an obvious choice of starting vertex.  Consequently, we first must arbitrarily choose a vertex in the feasible region as a starting point. 

As indicated above, our starting point is the y-intercept of the first constraint, which is the intersection of the active constraints $y=0$ and $5x+y=25$.  

Our starting vertex $(0,25)$ is on the _isoline_ ( or _isocline_ ) with level $k = 50$, which means it is a point on the level curve $4x+2y = 50$.

In [ ]:
fig, axes = plt.subplots()
add_method(axes, plotfill)

#plot the feasible region satisfying the constraint
x = np.linspace(0,20,20)
axes.plotfill( x, dict(a=5,b= 1,c=25), region = 'above', color = 'blue', label = '$5x+y \geq 25$' )
axes.plotfill( x, dict(a=3,b= 2,c=30), region = 'above', color = 'green', label = '$3x+2y \geq 30$' )
axes.plotfill( x, dict(a=1,b=-2,c=5), region = 'above', color = 'red', label = '$x - 2y \leq 5$')

# plot the level curve corresponding to the initial vertex 

# plot the initial vertex and the level curve containing it
for k in [50]:
    y = (k-4*x)/2
    axes.plot( x,y,color='magenta'  )
axes.text( x[2],y[2], ' $4x+2y = %s$' % k, color = 'magenta', fontsize = 14)
axes.scatter([0],[25], color = 'brown', zorder = 10)
    
# set axes properties
axes.set_xlim(0,)
axes.set_ylim(0,30)
axes.legend()

We want the objective to go _down_ in value, so we want the level curves to move _closer_ to the origin.  Thus, we next move to the intersection of the lines $5x+y = 25$ and $3x+2y = 30$, which is $\left(\tfrac{20}{7},\tfrac{75}{7}\right)$. 

This vertex $\left(\tfrac{20}{7},\tfrac{75}{7}\right)$ has level $k=\tfrac{230}{7} = 32.85714$, which means it is a point on the level curve $4x+2y = \tfrac{230}{7}$.

In [ ]:
for k in [44, 38, 32.85714]:
    y = (k-4*x)/2
    axes.plot( x,y,color='magenta'  )
axes.text( x[2],y[2], ' $4x+2y = %s$' % k, color = 'magenta', fontsize = 14)
axes.scatter([20/7],[75/7], color = 'brown', zorder = 10)
fig

Any level lower than $k = \tfrac{230}{7}$ corresponds to level curves that do not intersect the feasible region.  

Consequently, the answer is that there is a minimum of $z^* = \tfrac{230}{7}$ at the argmin of 
$$(x^*,y^*) = \left(\tfrac{20}{7},\tfrac{75}{7}\right).$$

<p>&nbsp;</p>

## Exercises 

Use the graphical method in a "simplex method" like form -- i.e., choosing a starting vertex and then moving from vertex to vertex until the level cannot be increased (or decreased if a minimization problem).  _From the graph of the Feasible region, determine how many vertices you did __not__ have to calculate because of this method_.  

1.  Find the maximum value of $z$ and the argmax of the following program: 
\begin{eqnarray*}
maximize \;  & z = 3x+y  \\
subject \; to \;   & x + y \le 8 \\
                   & \quad x \le 2 \\
                   & \quad y \le 2 \\
                   & x \ge 0, y \ge 0
\end{eqnarray*} 

2.  Find the maximum value of $z$ and the argmax of the following program: 
\begin{eqnarray*}
maximize \;  & z = x+y  \\
subject \; to \;   & 3x + 2y \le 12 \\
                   & 2x + 3y \le  9 \\
                   & \qquad y \le 2 \\
                   & x \ge 0, y \ge 0
\end{eqnarray*} 

3.  Find the maximum value of $z$ and the argmax of the following program: 
\begin{eqnarray*}
maximize \;  & z = x+y  \\
subject \; to \;   & \quad \;x + 2y  \le 4 \\
                   & \quad 4x + 2y \ge 12 \\
                   & -x + \;y \le 1 \\
                   & x \ge 0, y \ge 0
\end{eqnarray*} 

4.  Find the minimum value of $z$ and the argmin of the following program: 
\begin{eqnarray*}
maximize \;  & z = x+y  \\
subject \; to \;   & \; x + 2y  \ge 3 \\
                   & 2x + \; y  \ge 5 \\
                   & x \ge 0, y \ge 0
\end{eqnarray*} 

__You found all the linear programs for the following in the first set of exercises; and the vertices for their feasible regions in the second set of exercises.  Now apply the graphical method using our simplex-method like approach.__ 

5. A bakery sells pies and cakes.  Each pie costs \$2 to produce, requires a half an hour of labor, and sells for \$3; while each cake costs \$3, requires an hour of labor, and sells for \$5. How many pies and cakes, respectively, should be produced in each 8 hour day if the bakery spends no more than \$30 per day on supplies?   

6. A farmer will enclose a level rectangular field on 3 sides with a fence that costs \$5 per linear foot to build and \$1 per linear foot to maintain each year, but on the fourth side the field will be bounded by a wall that costs \$25 per linear foot to construct but is maintenance free after that.  What dimensions of the field will maximize its perimeter if the farmer wants to spend no more than \$10,000 total to enclose the field and \$1000 per year to maintain?   

7. Outdoor Furniture, Inc, makes benches and picnic tables. A bench requires 60 pounds of wood and requires 1 hour of labor.  A picnic table requires 80 pounds of wood and requires 2 hours of labor.  Their goal is to maximize the total number of benches and picnic tables produced per employee given that each employee is given 2000 pounds of wood each week and works 40 hours per week.  How many of each should they _decide_ to require of each employee? 

8. A farmer has $20$ acres on which he will plant $x$ acres of feed corn (for animals) and $y$ acres of sweet corn (for humans). It will cost \$1,000 per acre to grow feed corn and \$2,000 per acre to raise sweet corn; and the farmer has \$30,000 on hand at the beginning of the growing season.  Market analysis indicates that the number of acres of sweet corn should be no greater than twice the number of acres of feed corn, but he also wants to be taken seriously as a sweet corn producer, so he desires to grow no more than 15 acres of feed corn.  In units of thousands of dollars per acre, he knows that feed corn sells for \$2 thousands of dollars per acre, while sweet corn sells for \$5 thousands of dollars per acre. How many acres should he plant in feed corn, and how many in sweet corn, in order to maximize the revenue from the corn?  

9. Quality Furniture makes tables and chairs. The table below has the time required for each table and chair for cutting, assembly, and finishing, with the bottom row the minutes available each day for each (480 minutes = 8 hours).  

|     Item        | Cutting | Assembly | Finishing | profit |
|:---------------:|:-------:|:--------:|:---------:| ------:|
|    Table        |    60   |    45    |    48     |   \$90 |
|    Chair        |    20   |    30    |    24     |   \$10 |
| --------------- | ------- | -------- | --------- | ------ |
|   Available     |   480   |   480    |    480    |        | 

<div style = "margin-left:25px">Also, Quality Furniture needs  to make at least 4 chairs for each table.  The profit from the sale of each is shown in the final column.  How many tables and chairs, respectively, should they decide to make each day in order to maximize profit? </div>